In [1]:
import pandas as pd
import math as math
from numpy import sqrt
from numpy import asarray
from pandas import read_csv
from tensorflow.keras import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from scipy.special import boxcox, inv_boxcox

import numpy as np
import tensorflow as tf
from sklearn.model_selection import TimeSeriesSplit


In [2]:
data = pd.read_csv('https://download.data.world/s/o5bwxhou5qf7ym2xl4febbecw56rue')

In [3]:
data

,date,country_region,sum
0,2020-01-23,China,643
1,2020-01-24,China,920
2,2020-01-25,China,1406
3,2020-01-26,China,2075
4,2020-01-27,China,2877
...,...,...,...
119,2020-03-20,"Korea, South",8652
120,2020-03-21,"Korea, South",8799
121,2020-03-22,"Korea, South",8961
122,2020-03-23,"Korea, South",8961


In [26]:
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')

In [27]:
us_data = data.loc[data['country_region'] == 'US']
korea_data = data.loc[data['country_region'] == 'Korea, South']
china_data = data.loc[data['country_region'] == 'China']

In [28]:
[korea_data, china_data]

[           country_region   sum
 date                           
 2020-01-23   Korea, South     1
 2020-01-24   Korea, South     2
 2020-01-25   Korea, South     2
 2020-01-26   Korea, South     3
 2020-01-27   Korea, South     4
 2020-01-28   Korea, South     4
 2020-01-29   Korea, South     4
 2020-01-30   Korea, South     4
 2020-01-31   Korea, South    11
 2020-02-01   Korea, South    12
 2020-02-02   Korea, South    15
 2020-02-03   Korea, South    15
 2020-02-04   Korea, South    16
 2020-02-05   Korea, South    19
 2020-02-06   Korea, South    23
 2020-02-07   Korea, South    24
 2020-02-08   Korea, South    24
 2020-02-09   Korea, South    25
 2020-02-10   Korea, South    27
 2020-02-11   Korea, South    28
 2020-02-12   Korea, South    28
 2020-02-13   Korea, South    28
 2020-02-14   Korea, South    28
 2020-02-15   Korea, South    28
 2020-02-16   Korea, South    29
 2020-02-17   Korea, South    30
 2020-02-18   Korea, South    31
 2020-02-19   Korea, South    31
 2020-02-2

In [33]:
## Standardization
def normalizeData(data):
    scaler = MinMaxScaler().fit(data)
    data = scaler.transform(data)
    return data, scaler

def split_sequence(sequence, n_steps):
    len_seq = sequence.shape[1]
    X, y = list(), list()
    for i in range(len_seq):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len_seq-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[:, i:end_ix], sequence[:, end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return asarray(X), asarray(y)

In [35]:
confirmed_cases = np.asarray([korea_data['sum'], china_data['sum']])

In [36]:
n_steps = 1
X, y = split_sequence(confirmed_cases, n_steps)

In [37]:
X[1:30]

array([[[    2],
        [  920]],

       [[    2],
        [ 1406]],

       [[    3],
        [ 2075]],

       [[    4],
        [ 2877]],

       [[    4],
        [ 5509]],

       [[    4],
        [ 6087]],

       [[    4],
        [ 8141]],

       [[   11],
        [ 9802]],

       [[   12],
        [11891]],

       [[   15],
        [16630]],

       [[   15],
        [19716]],

       [[   16],
        [23707]],

       [[   19],
        [27440]],

       [[   23],
        [30587]],

       [[   24],
        [34110]],

       [[   24],
        [36814]],

       [[   25],
        [39829]],

       [[   27],
        [42354]],

       [[   28],
        [44386]],

       [[   28],
        [44759]],

       [[   28],
        [59895]],

       [[   28],
        [66358]],

       [[   28],
        [68413]],

       [[   29],
        [70513]],

       [[   30],
        [72434]],

       [[   31],
        [74211]],

       [[   31],
        [74619]],

       [[  104],
        [75

In [38]:
## Splitting into validation and test sets
number_of_rows = X.shape[0]
n_train = math.ceil(number_of_rows*80/100)
n_test = n_train + math.floor(number_of_rows*10/100) 
X_train, y_train, X_val, y_val, X_test, y_test = X[:n_train], y[:n_train], X[n_train:n_test], y[n_train:n_test], X[n_test:], y[n_test:]

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=X.shape, return_state=True))
model.add(Dense(3, input_shape=))